In [95]:
import numpy as np
from scipy.integrate import solve_ivp
import pandas as pd

Fixed Model Parameters

In [96]:
alpha_1 = 3e-4
alpha_2 = 1.2
fk_max = 9
R_k = 0.01
v_1 = 3e2
v_2 = 14.4 *1e-1
"""
alpha_1 = 1e-3
alpha_2 = 1.44
fk_max = 1
R_k = 0.5
mu = 2
v_1 = 1.2e2
v_2 = 14.4 
"""

'\nalpha_1 = 1e-3\nalpha_2 = 1.44\nfk_max = 1\nR_k = 0.5\nmu = 2\nv_1 = 1.2e2\nv_2 = 14.4 \n'

Kill Function and Dimensionless ODE Model

In [97]:
def kill_function(R):
     f = fk_max*(R/(R + R_k))

     return f

def model(t, z, gamma, mu):

    dAdt = -alpha_1*(1 - z[1])*z[0]*z[2] + alpha_2*gamma*z[1]*z[2]
    dRdt = (alpha_1/gamma)*(1 - z[1])*z[0] - alpha_2*z[1]
    dSdt = -kill_function(z[1])*z[3]
    dCdt = v_1*(mu - z[3])*(z[2] - z[3]) - v_2*z[3] - kill_function(z[1])*z[3]
    dzdt = [dAdt, dRdt, dSdt, dCdt]

    return dzdt

ICs and Simulation Parameters

In [98]:
z0 = [1, 0, 1, 0] 
t = np.arange(0, 1.1, 0.1)
t_span = [0, 1]
mu_vals = [1.25, 2.5, 5, 10, 20]
gamma_vals = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
noise_vals = [0.01, 0.05, 0.1, 0.2]

Create Dataframe

In [99]:
df = pd.DataFrame(columns=['t', 'mu', 'gamma', 'noise level', 'A', 'R', 'S', 'C'])


In [100]:

for gamma in gamma_vals:
    for mu in mu_vals:
        for noise_level in noise_vals:
            j=0
            while j<10:
                
                z = solve_ivp(model, t_span, z0, args = (gamma, mu), method ='Radau', t_eval = t)

                # add noise
                A_array = z.y[0]
                R_array = z.y[1]
                S_array = z.y[2]
                C_array = z.y[3]

                for i in range(len(A_array)):
                    A_array[i] = A_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                    R_array[i] = R_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                    S_array[i] = S_array[i]*(1 + np.random.uniform(-noise_level, noise_level))
                    C_array[i] = C_array[i]*(1 + np.random.uniform(-noise_level, noise_level))

                # add data to dataframe
                for i in range(len(A_array)):
                    df = df.append({'t' : t[i],
                                    'mu' : mu,
                                    'gamma' : gamma,
                                    'noise level' : noise_level,
                                    'A' : A_array[i],
                                    'R' : R_array[i],
                                    'S' : S_array[i],
                                    'C' : C_array[i]}, ignore_index=True)
                
                j += 1

        

In [101]:
print(df)

         t     mu     gamma  noise level         A         R         S  \
0      0.0   1.25  1.000000         0.01  1.004856  0.000000  1.006123   
1      0.1   1.25  1.000000         0.01  0.993690  0.000028  0.997054   
2      0.2   1.25  1.000000         0.01  1.006935  0.000053  1.001066   
3      0.3   1.25  1.000000         0.01  0.999702  0.000076  0.998657   
4      0.4   1.25  1.000000         0.01  1.007558  0.000095  0.976934   
...    ...    ...       ...          ...       ...       ...       ...   
15395  0.6  20.00  0.000001         0.20  0.846756  0.820303  0.004758   
15396  0.7  20.00  0.000001         0.20  0.968643  1.162470  0.001811   
15397  0.8  20.00  0.000001         0.20  0.912718  1.165655  0.000865   
15398  0.9  20.00  0.000001         0.20  1.142071  0.920668  0.000357   
15399  1.0  20.00  0.000001         0.20  1.154732  1.174828  0.000155   

              C  
0      0.000000  
1      0.977494  
2      0.986133  
3      0.968875  
4      0.966777  
...

Write Dataframe to CSV

In [102]:
df.to_csv('basic_model_synthetic_data.csv')